
# Algorithmes d'Optimisation dans le Cas Convexe

Ce notebook explore différentes méthodes d'optimisation appliquées à un problème de régression linéaire.
Nous allons implémenter et comparer plusieurs algorithmes d'optimisation gradient-based :

- **Gradient Descent (GD)**
- **Stochastic Gradient Descent (SGD)**
- **SAGA (Stochastic Average Gradient Algorithm)**
- **SVRG (Stochastic Variance Reduced Gradient)**

Nous considérons le problème d'optimisation suivant :

\[ \min_{w} \frac{1}{2} || A w - b ||^2 \]

où :

- \( A \in \mathbb{R}^{n \times d} \) est la matrice des caractéristiques.
- \( b \in \mathbb{R}^{n} \) est le vecteur des valeurs cibles.
- \( w \in \mathbb{R}^{d} \) représente les paramètres à optimiser.

Chaque cellule de code est précédée d'une explication et des formules mathématiques associées.


## 1️⃣ Importation des bibliothèques nécessaires

In [ ]:

import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable, grad
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import matplotlib.pyplot as plt
import numpy as np
import time
import copy

torch.manual_seed(0)



## 2️⃣ Initialisation des paramètres

Nous définissons une fonction pour initialiser les paramètres d'un modèle avec une valeur spécifique.


In [ ]:

def initialize_para(nn, val):
    for para in nn.parameters():
        para.data.fill_(val)
    return nn



## 3️⃣ Définition du modèle de régression linéaire

Nous implémentons un modèle de régression linéaire basé sur PyTorch. Ce modèle suit la fonction :

\[ f(w) = A w \]

Nous définissons également plusieurs méthodes d'optimisation :

- **Calcul du gradient complet**.
- **Gradient partiel pour les méthodes stochastiques**.
- **Méthodes d'optimisation GD, SGD, SAGA, et SVRG**.


In [ ]:

class Model(nn.Module):
    def __init__(self, n_samples, n_features):
        super(Model, self).__init__()
        self.n_samples = n_samples
        self.n_features = n_features
        self.A = Variable(torch.randn(n_samples, n_features), requires_grad=False)
        self.b = Variable(torch.randn(n_samples, 1), requires_grad=False)
        self.w = nn.Parameter(torch.zeros(n_features, 1), requires_grad=True)

    def forward(self):
        return torch.mm(self.A, self.w)

    def full_loss_grad(self, criterion):
        grad = 0
        output = self.forward()
        loss = criterion(output, self.b)
        loss.backward()
        for para in self.parameters():
            grad += para.grad.data.norm(2) ** 2
        return loss.data.item(), (1.0 / self.n_samples) * np.sqrt(grad)



## 4️⃣ Descente de Gradient (GD)

L'algorithme **Gradient Descent** suit la mise à jour :

\[ w^{(t+1)} = w^{(t)} - \eta \nabla L(w) \]

où \( \eta \) est le taux d'apprentissage.


In [ ]:

def gd_backward(self, criterion, max_iter, lr):
    total_loss_epoch = [0 for _ in range(max_iter)]
    grad_norm_epoch = [0 for _ in range(max_iter)]
    for epoch in range(max_iter):
        self.zero_grad()
        total_loss_epoch[epoch], grad_norm_epoch[epoch] = self.full_loss_grad(criterion)
        for i_data in range(self.n_samples):
            self.zero_grad()
            _ = self.grad(criterion)
            for para in self.parameters():
                para.data -= lr * para.grad.data
    return total_loss_epoch, grad_norm_epoch



## 5️⃣ Descente de Gradient Stochastique (SGD)

L'algorithme **SGD** met à jour les poids après chaque échantillon aléatoire :

\[ w^{(t+1)} = w^{(t)} - \eta \nabla L_i(w) \]

où \( \nabla L_i(w) \) est le gradient basé sur un seul échantillon.


In [ ]:

def sgd_backward(self, criterion, max_iter, lr):
    total_loss_epoch = [0 for _ in range(max_iter)]
    grad_norm_epoch = [0 for _ in range(max_iter)]
    for epoch in range(max_iter):
        self.zero_grad()
        total_loss_epoch[epoch], grad_norm_epoch[epoch] = self.full_loss_grad(criterion)
        for i_data in range(self.n_samples):
            self.zero_grad()
            _, loss = self.partial_grad(criterion)
            for para in self.parameters():
                para.data -= lr * para.grad.data
    return total_loss_epoch, grad_norm_epoch



## 6️⃣ Méthode SAGA

L'algorithme **SAGA** corrige la variance du gradient en utilisant une mémoire des gradients passés.


In [ ]:

def saga_backward(self, criterion, n_epoch, learning_rate):
    grad_norm_epoch = [0 for _ in range(n_epoch)]
    total_loss_epoch = [0 for _ in range(n_epoch)]
    mean_grad = copy.deepcopy(self)
    prev_stoc_grad = [copy.deepcopy(self) for _ in range(self.n_samples)]
    epoch = 0
    for i in range(n_epoch * self.n_samples):
        if i % self.n_samples == 0:
            self.zero_grad()
            total_loss_epoch[epoch], grad_norm_epoch[epoch] = self.full_loss_grad(criterion)
            epoch += 1
        self.zero_grad()
        i_data, loss = self.partial_grad(criterion)
        for para, para_prev, para_mean in zip(self.parameters(), prev_stoc_grad[i_data].parameters(), mean_grad.parameters()):
            saga_update = para.grad.data - para_prev.data + para_mean.data
            para_mean.data += (1.0 / self.n_samples) * (para.grad.data - para_prev.data)
            para_prev.data = para.grad.data.clone()
            para.data.sub_(saga_update * learning_rate)
    return total_loss_epoch, grad_norm_epoch
